# Spotify Web Scrapping w Python

### Scrapping from wikipedia

In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Access HTML
url = 'https://en.wikipedia.org/wiki/List_of_most-streamed_songs_on_Spotify'
headers = {'User-Agent': 'MyWebScrapper/1.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of Spotify streaming records - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-conten

In [3]:
# Subset desired most streamed songs table
tables = soup.find_all('table', class_='wikitable')
most_streamed_songs = tables[0]
most_streamed_songs.find_all('th')[:5]

[<th>Rank
 </th>,
 <th>Song
 </th>,
 <th>Artist(s)
 </th>,
 <th>Streams<br/>(billions)
 </th>,
 <th>Release date
 </th>]

In [4]:
# Extract table headers into table_headers
header_cells = most_streamed_songs.find_all('tr')[0]
table_headers = [cell.get_text(strip=True) for cell in header_cells if cell.get_text(strip=False) != '\n']
table_headers

['Rank', 'Song', 'Artist(s)', 'Streams(billions)', 'Release date', 'Ref.']

In [5]:
# Extract table rows into a table_data
rows = most_streamed_songs.find_all('tr')[1:]

table_data = []
for row in rows:
    song_name = row.find('th').get_text(strip=True).replace('"', '')
    cells = row.find_all('td')

    tmp_content = [cell.get_text(strip=True) for cell in cells]
    tmp_content.insert(1, song_name)

    table_data.append(tmp_content)
    
table_data[:5]

[['1', 'Blinding Lights', 'The Weeknd', '4.995', '29 November 2019', '[1]'],
 ['2', 'Shape of You', 'Ed Sheeran', '4.524', '6 January 2017', '[2]'],
 ['3',
  'Starboy',
  'The WeekndandDaft Punk',
  '4.058',
  '21 September 2016',
  '[3]'],
 ['4',
  'Someone You Loved',
  'Lewis Capaldi',
  '4.028',
  '8 November 2018',
  '[4]'],
 ['5', 'As It Was', 'Harry Styles', '4.017', '1 April 2022', '[5]']]

In [51]:
df_raw = pd.DataFrame(data=table_data, columns=table_headers)
df_raw.head()

,Rank,Song,Artist(s),Streams(billions),Release date,Ref.
0,1,Blinding Lights,The Weeknd,4.995,29 November 2019,[1]
1,2,Shape of You,Ed Sheeran,4.524,6 January 2017,[2]
2,3,Starboy,The WeekndandDaft Punk,4.058,21 September 2016,[3]
3,4,Someone You Loved,Lewis Capaldi,4.028,8 November 2018,[4]
4,5,As It Was,Harry Styles,4.017,1 April 2022,[5]


### Data preprocessing and analysis

In [69]:
df_baking = df_raw.copy()
df_baking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Rank               101 non-null    object
 1   Song               100 non-null    object
 2   Artist(s)          100 non-null    object
 3   Streams(billions)  100 non-null    object
 4   Release date       100 non-null    object
 5   Ref.               100 non-null    object
dtypes: object(6)
memory usage: 4.9+ KB


In [70]:
display(df_baking[df_baking.isna().any(axis=1)]) # This is the last line of the wikipedia table, thus will be dropped
df_baking = df_baking.dropna()

,Rank,Song,Artist(s),Streams(billions),Release date,Ref.
100,As of 28 August 2025,None,None,None,None,None


In [71]:
df_baking.columns = df_baking.columns.str.lower()

### Cleaning and dtypes

- Rank column can be dropped, since the rank can be implicitly known by sorting the values by # of streams
- Songs $\rightarrow$ str
- Artists $\rightarrow$ category (Only a finite number of artist for different songs)
- Release date $\rightarrow$ datetime (extract year)
- Ref column can be dropped, since they refer to an external link inside wikipedia, which is not useful for analysis

In [72]:
# Drop unnecesary columns
df_baking = df_baking.drop(columns=['rank', 'ref.'])

In [73]:
# Converting str columns
df_baking['song'] = df_baking['song'].astype('str')

# Cleaning artist names 
df_baking["artist(s)"] = df_baking["artist(s)"].str.replace("and", " and ", regex=False)
df_baking["artist(s)"] = df_baking["artist(s)"].str.replace("with", " with ", regex=False)
df_baking['artist(s)'] = df_baking['artist(s)'].astype('category')

In [74]:
# Converting release into date dtype
df_baking['release date'] = pd.to_datetime(df_baking['release date']).dt.year
df_baking['release date']

0     2019
1     2017
2     2016
3     2018
4     2022
      ... 
95    2007
96    2003
97    2017
98    2017
99    1988
Name: release date, Length: 100, dtype: int32

In [75]:
df = df_baking.copy()
df.sample(10)

,song,artist(s),streams(billions),release date
12,Lovely,Billie Eilish and Khalid,3.436,2018
20,Rockstar,Post Malone and 21 Savage,3.234,2017
6,Sunflower,Post Malone and Swae Lee,3.932,2018
33,Viva La Vida,Coldplay,2.932,2008
73,Don't Stop Believin',Journey,2.550,1981
54,That's What I Like,Bruno Mars,2.749,2016
90,Uptown Funk,Mark Ronson and Bruno Mars,2.418,2014
76,One Kiss,Calvin Harris and Dua Lipa,2.526,2018
5,Sweater Weather,The Neighbourhood,3.978,2012
14,Say You Won't Let Go,James Arthur,3.371,2016


### Visualization